# Core

> The building blocks to the UI

## Imports

In [ ]:
#| default_exp core

In [ ]:
#| hide
#| export
from fasthtml.common import *
from fasthtml.svg import Svg

In [ ]:
from nbdev.showdoc import show_doc
from IPython.display import HTML

## Utils

In [ ]:
#| export
# need a better name, stringify might be too general for what it does 
def stringify(o # String, Tuple, or Enum options we want stringified
             ): # String that can be passed FT comp args (such as `cls=`)
    "Converts input types into strings that can be passed to FT components"  
    if is_listy(o): return ' '.join(map(str,o)) if o else ""
    return o.__str__()

In [ ]:
assert stringify('abc') == 'abc'
assert stringify(('abc','def')) == 'abc def'
assert 'uk-input ' + stringify(()) == 'uk-input '
assert 'uk-input ' + stringify("") == 'uk-input '

### Components

In [ ]:
#| export
def UkGenericComponent(component_fn, *c, cls=(), **kwargs): return component_fn(cls=cls, **kwargs)(*c)

In [ ]:
#| export
def H1(*c, cls=(), **kwargs): return UkGenericComponent(H1, *c, cls=('uk-h1',stringify(cls)), **kwargs)
def H2(*c, cls=(), **kwargs): return UkGenericComponent(H2, *c, cls=('uk-h2',stringify(cls)), **kwargs)
def H3(*c, cls=(), **kwargs): return UkGenericComponent(H3, *c, cls=('uk-h3',stringify(cls)), **kwargs)
def H4(*c, cls=(), **kwargs): return UkGenericComponent(H4, *c, cls=('uk-h4',stringify(cls)), **kwargs)
def H5(*c, cls=(), **kwargs): return UkGenericComponent(H5, *c, cls=('uk-h5',stringify(cls)), **kwargs)
def H6(*c, cls=(), **kwargs): return UkGenericComponent(H6, *c, cls=('uk-h6',stringify(cls)), **kwargs)

In [ ]:
#| export
class ButtonT(VEnum):
    default   = 'default'
    primary   = 'primary'
    secondary = 'secondary'
    danger    = 'danger'
    ghost     = 'ghost'
    text      = 'text'
    link      = 'link'

NameError: name 'VEnum' is not defined

In [ ]:
#| export
def Button(*c, # Content for the button
            cls=ButtonT.default, # Classes for the button
            **kwargs # Additional args for the button
            ):
    "Creates a button with uk styling"
    cls = stringify(cls)    
    return Button(type='button', cls=('uk-button',cls), **kwargs)(*c)

NameError: name 'ButtonT' is not defined

In [ ]:
UkButton('MyButton', cls=UkButtonT.primary)

```html
<button type="button" class="uk-button uk-button-primary">MyButton</button>
```

In [ ]:
#| export
def UkIconButton(*c, sz='small', cls=(), **kwargs):
    "Creates an `IconButton` with uk styling"
    if sz not in ('small','medium','large'): raise ValueError(f"Invalid size '{sz}'. Must be 'small', 'medium', or 'large'.")
    return Button(cls=f'uk-icon-button uk-icon-button-{sz} ' + stringify(cls), **kwargs)(*c)

In [ ]:
UkIconButton(UkIcon('home'))

```html
<button class="uk-icon-button uk-icon-button-small "><span uk-icon="icon: home; ratio: 1"></span></button>
```

## Drop Downs

In [ ]:
#| export
def Options(*c,                    # Content for an `Option`
            selected_idx:int=None, # Index location of selected `Option`
            disabled_idxs:set=None # Idex locations of disabled `Options`
           ):
    "Helper function to wrap things into `Option`s for use in `UKSelect`"
    return [Option(o,selected=i==selected_idx, disabled=disabled_idxs and i in disabled_idxs) for i,o in enumerate(c)]

In [ ]:
list(map(to_xml,Options('option1','option2','option3', selected_idx=1, disabled_idxs={0,2})))

['<option disabled>option1</option>',
 '<option selected>option2</option>',
 '<option disabled>option3</option>']

In [ ]:
#| export
def UkSelect(*option,            # Options for the select dropdown (can use `Options` helper function to create)
             label=(),           # String or FT component for the label
             lbl_cls=(),         # Additional classes for the label
             inp_cls=(),         # Additional classes for the select input
             cls=('space-y-2',), # Classes for the outer div
             id="",              # ID for the select input
             name="",            # Name attribute for the select input
             placeholder="",     # Placeholder text for the select input
             searchable=False,   # Whether the select should be searchable
             **kwargs):          # Additional arguments passed to Uk_select
    "Creates a select dropdown with uk styling"
    lbl_cls, inp_cls, cls = map(stringify, (lbl_cls, inp_cls, cls))
    if label: 
        lbl = Label(cls=f'uk-form-label {lbl_cls}', fr=id)(label) 
    select = Uk_select(*option, cls=inp_cls, uk_cloak=True, id=id, 
                       name=name, placeholder=placeholder, searchable=searchable, **kwargs)
    return Div(cls=cls)(lbl, select) if label else Div(cls=cls)(select)

In [ ]:
UkSelect(*Options('a', 'b'), label='MyLabel', placeholder='MyPlaceholder')

```html
<div class="space-y-2">
<label class="uk-form-label ">MyLabel</label><uk-select uk-cloak placeholder="MyPlaceholder"><option>a</option><option>b</option></uk-select></div>

```

In [ ]:
#| exporti
def _UkDropdownButtonOptions(opt_grps, opt_hdrs=None):
    res = []
    for g,h in zip_longest(opt_grps, tuplify(opt_hdrs)):
        if h: res.append(Li(cls="uk-nav-header")(h if isinstance(h,FT) else Div(h)))
        if isinstance(g,(list,tuple)): res += list(map(Li, g))
        else: res.append(Li(g))
    return res

In [ ]:
#| export
def UkDropdownButton(
    *opt_grp,        # List of options to be displayed in the dropdown
    opt_hdrs=None,  # List of headers for each option group, or None
    label=None,     # String, FT component, or None for the `Button`
    btn_cls=UkButtonT.default,  # Button class(es)
    cls=(),         # Parent div class
    dd_cls=(),      # Class that goes on the dropdown container
    icon='triangle-down',  # Icon to use for the dropdown
    icon_cls='',    # Additional classes for the icon
    icon_position='right'  # Position of the icon: 'left' or 'right'
    ):
    dd_cls, btn_cls, cls, icon_cls = map(stringify, (dd_cls, btn_cls, cls, icon_cls))
    icon_component = UkIcon(icon, cls=icon_cls) if icon else None
    btn_content = [] if label is None else [label]
    if icon_component: btn_content.insert(0 if icon_position == 'left' else len(btn_content), icon_component)
    btn = Button(type='button', cls='uk-button ' + btn_cls)(*btn_content)
    dd = Div(uk_dropdown='mode: click; pos: bottom-right', cls='uk-dropdown uk-drop ' + dd_cls)(
        Ul(cls='uk-dropdown-nav')(*_UkDropdownButtonOptions(opt_grp, opt_hdrs)))
    return Div(cls=cls)(Div(cls='flex items-center space-x-4')(btn, dd))

In [ ]:
# UkDropdownButton((A('Option 1.1', _href='#'), A('Option 1.2', _href='#')), 
#                  (A('Option 2.1', _href='#'), A('Option 2.2', _href='#')),
#                  opt_hdrs=('Header 1', 'Header 2'))

## Headings

In [ ]:
#| export
def UkGenericComponent(component_fn, *c, cls=(), **kwargs):
    res = component_fn(cls=cls, **kwargs)(*c)
    return res

In [ ]:
#| export
def UkH1(*c, cls=(), **kwargs): return UkGenericComponent(H1, *c, cls=('uk-h1',stringify(cls)), **kwargs)
def UkH2(*c, cls=(), **kwargs): return UkGenericComponent(H2, *c, cls=('uk-h2',stringify(cls)), **kwargs)
def UkH3(*c, cls=(), **kwargs): return UkGenericComponent(H3, *c, cls=('uk-h3',stringify(cls)), **kwargs)
def UkH4(*c, cls=(), **kwargs): return UkGenericComponent(H4, *c, cls=('uk-h4',stringify(cls)), **kwargs)
def UkH5(*c, cls=(), **kwargs): return UkGenericComponent(H5, *c, cls=('uk-h5',stringify(cls)), **kwargs)
def UkH6(*c, cls=(), **kwargs): return UkGenericComponent(H6, *c, cls=('uk-h6',stringify(cls)), **kwargs)

In [ ]:
UkH1('My Uk styled header',cls='another-class')

```html
<h1 class="uk-h1 another-class">My Uk styled header</h1>

```

## Dividers

In [ ]:
#| export
def UkHSplit(*c, cls=(), line_cls=(), text_cls=()):
    cls, line_cls, text_cls = map(stringify,(cls, line_cls, text_cls))
    return Div(cls='relative ' + cls)(
        Div(cls="absolute inset-0 flex items-center " + line_cls)(Span(cls="w-full border-t border-border")),
        Div(cls="relative flex justify-center " + text_cls)(Span(cls="bg-background px-2 ")(*c)))

In [ ]:
#| export
def UkHLine(lwidth=2, y_space=4): return Div(cls=f"my-{y_space} h-[{lwidth}px] w-full bg-secondary")

In [ ]:
#| export
def UkNavDivider(): return Li(cls="uk-nav-divider")

## NavBar

In [ ]:
#| export
def UkNavbarDropdown(*c, label, href='#', cls='', has_header=False, **kwargs):
    fn = lambda x: Li(item, cls='uk-drop-close', href='#demo', uk_toggle=True)
    flattened = []
    for i, item in enumerate(c):
        if i > 0: flattened.append(Li(cls="uk-nav-divider"))
        if isinstance(item, (list,tuple)): flattened.extend(map(Li, item))
        else: flattened.append(Li(item, cls="uk-nav-header" if i == 0 and has_header else None, uk_toggle=True))
    return (Li(cls=cls, **kwargs)(
                A(label, cls='uk-drop-close', href='#', uk_toggle=True), 
                Div(cls='uk-navbar-dropdown', uk_dropdown="mode: click; pos: bottom-left")(Ul(cls='uk-nav uk-dropdown-nav')(*flattened))))

In [ ]:
#| export
def _NavBarSide(n, s):
    def add_class(item):
        if isinstance(item, str): return Li(cls='uk-navbar-item')(item)
        else: item.attrs['class'] = f"{item.attrs.get('class', '')} uk-navbar-item".strip()
        return item
    return Div(cls=f'uk-navbar-{s}')(Ul(cls='uk-navbar-nav')(*map(add_class, tuplify(n))))

In [ ]:
#| export
def UkNavbar(lnav: Sequence[Union[str, FT]]=None, 
             rnav: Sequence[Union[str, FT]]=None, 
             cls='',
             **kwargs
            ) -> FT:
    return Div(cls='uk-navbar-container uk-width-1-1 relative z-10 '+ stringify(cls), uk_navbar=True, **kwargs)(
             _NavBarSide(lnav,'left') if lnav else '',
             _NavBarSide(rnav,'right') if rnav else '')

## Sidebar

In [ ]:
sidebar_items = ["Profile", "Account", "Appearance", "Notifications", "Display"]
tst = Ul(cls='uk-nav-primary', uk_switcher="connect: #component-nav; animation: uk-animation-fade")(
                            *map(lambda x: Li(A(x)),sidebar_items))

In [ ]:
#| export
def UkSidebar(*ul,                 # Each Ul can be it's own section.  Use A for links!
              cls='space-y-4 p-4', # Classes for outer container
              **kwargs             # Kwargs for outer container
             ):
    "Creates a styled sidebar component"
    styles = ('uk-nav-default', 'uk-nav-primary','uk-nav-secondary')
    sidebar = []
    for section in tuplify(ul):
        section = copy.deepcopy(section)
        _sattrs = section.attrs
        if 'class' not in _sattrs: _sattrs['class'] = ''
        if 'uk-nav' not in _sattrs: _sattrs['class'] += ' uk-nav '
        if not any(x in styles for x in _sattrs['class'].split()): _sattrs['class'] += ' uk-nav-default '
        sidebar.append(section)  
    return Div(cls=cls, **kwargs)(*sidebar)

In [ ]:
sidebar_items = ["Profile", "Account"]
tst = Ul(cls='uk-nav-primary')(*map(lambda x: Li(A(x)),sidebar_items))
UkSidebar(tst)

```html
<div class="space-y-4 p-4">
  <ul class="uk-nav-primary uk-nav ">
    <li>
<a href="#">Profile</a>    </li>
    <li>
<a href="#">Account</a>    </li>
  </ul>
</div>

```

## Navtab

In [ ]:
#| export
def NavTab(text, active=False):
    return Li(cls="uk-active" if active else " ")(A(text, href="#demo", uk_toggle=True))

def UkTab(*items, maxw=96, cls='', **kwargs):
    cls = stringify(cls)
    return Ul(cls=f"uk-tab-alt max-w-{maxw} "+cls,**kwargs)(*[NavTab(item, active=i==0) for i, item in enumerate(items)])

## Cards

In [ ]:
#|export
def Card(*c, # Components that go in the body
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls=(), # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res += [Div(cls='uk-card-header ' + header_cls)(header),]
    res += [Div(cls='uk-card-body ' + body_cls)(*c),]
    if footer: res += [Div(cls='uk-card-footer ' + footer_cls)(footer),]
    return Div(cls='uk-card '+cls, **kwargs)(*res)

## Modals

In [ ]:
#| export
def UkModalTitle(*c, cls=()): return Div(cls='uk-modal-title ' + stringify(cls))(*c)

def Modal(*c,
        header=None, # Components that go in the header
        footer=None,  # Components that go in the footer
        body_cls='space-y-6', # classes for the body
        header_cls='p-6', # classes for the header
        footer_cls=(), # classes for the footer
        cls=(), #class for outermost component
        **kwargs # classes that for the card itself
        ):
    header_cls, footer_cls, body_cls, cls = map(stringify, (header_cls, footer_cls, body_cls, cls))
    res = []
    if header: res += [Div(cls='uk-modal-header ' + header_cls)(header),]
    res += [Div(cls='uk-modal-body uk-modal-dialog ' + body_cls)(*c),]
    if footer: res += [Div(cls='uk-modal-footer ' + footer_cls)(footer),]
    return Div(cls='uk-modal uk-modal-container' + cls, uk_modal=True, **kwargs)(*res)


## Tables

In [ ]:
#| export
def _default_cell(col, row): return Td(row[col], cls='p-2')

In [ ]:
def render_header(col):
    cls = 'p-2 ' + 'uk-table-shrink' if col in ('Done','Actions') else ''
    return Th(col, cls=cls)

def render_cell(col, row):
    match col:
        case "ColA":  return Td(row[col], cls=TextT.muted_sm)
        case "ColB":  return Td(row[col], cls=TextT.medium_sm)
        case 'ColC':  return Td(row[col])
        case 'ColD':  return Td(row[col], cls='p-2')
        
cols = ('ColA', 'ColB', 'ColC', 'ColD')
data = [{'ColA':1, 'ColB':2, 'ColC':3, 'ColD':4},]
# UkTable(cols, data, cell_render=render_cell, header_render=render_header)

In [ ]:
#| export
def TableHeader(columns, header_render=None):
    rndr = header_render or Th(cls='p-2')
    return Tr(*map(rndr, columns))

In [ ]:
#| export
def TableRow(row, columns, cell_render=None):
    rndr = cell_render or _default_cell
    return Tr(*[rndr(col, row) for col in columns])

In [ ]:
#| export
def UkTable(columns, data, *args, cls=(), footer=None, cell_render=None, header_render=None, **kwargs):
    # table middle, small : Should these be parameterized?
    # Document, especially cell and header render
    table_cls = 'uk-table uk-table-middle uk-table-divider uk-table-hover uk-table-small ' + stringify(cls)
    cell_render   = ifnone(cell_render,   lambda c,r: Td(cls='p-2')(r[c]))
    header_render = ifnone(header_render, lambda c:   Th(cls='p-2')(c))
    
    head = Thead(Th(*map(header_render, columns)))
    
    table_content = []
    for row in data: table_content.append(Tr(*[cell_render(col, row) for col in columns]))
        
    body = Tbody(*table_content)
    if footer: table_content.append(Tfoot(footer))
    return Table(cls=table_cls, *args, **kwargs)(*[head,body])

In [ ]:
UkTable(cols,data)

```html
<table class="uk-table uk-table-middle uk-table-divider uk-table-hover uk-table-small ">
  <thead>
    <th>
      <th class="p-2">ColA</th>
      <th class="p-2">ColB</th>
      <th class="p-2">ColC</th>
      <th class="p-2">ColD</th>
    </th>
  </thead>
  <tbody>
    <tr>
      <td class="p-2">1</td>
      <td class="p-2">2</td>
      <td class="p-2">3</td>
      <td class="p-2">4</td>
    </tr>
  </tbody>
</table>

```

In [ ]:
%%aip

How do I fix that error?

In [ ]:
# Let's inspect the data structure to understand why we're getting this error
print("Columns:", cols)
print("\nData:")
for item in data:
    print(item)

# Let's also check the type of each item in data
print("\nTypes in data:")
for item in data:
    print(type(item))

# If data contains dictionaries as expected, let's try to access a key
if data and isinstance(data[0], dict):
    print("\nFirst item keys:", list(data[0].keys()))
    print("First item values:", list(data[0].values()))
else:
    print("\nData is not in the expected format (list of dictionaries)")

# Let's modify the UkTable function to handle both dictionary and non-dictionary data
def UkTable(columns, data, *args, cls=(), footer=None, cell_render=None, header_render=None, **kwargs):
    table_cls = 'uk-table uk-table-middle uk-table-divider uk-table-hover uk-table-small ' + stringify(cls)
    
    if data and isinstance(data[0], dict):
        cell_render = ifnone(cell_render, lambda c, r: Td(cls='p-2')(r[c]))
    else:
        cell_render = ifnone(cell_render, lambda c, r: Td(cls='p-2')(r))
    
    header_render = ifnone(header_render, lambda c: Th(cls='p-2')(c))
    
    head = Thead(Tr(*map(header_render, columns)))
    
    if data and isinstance(data[0], dict):
        body = Tbody(*[Tr(*map(lambda c: cell_render(c, row), columns)) for row in data])
    else:
        body = Tbody(*[Tr(*map(lambda val: cell_render(None, val), row)) for row in data])
    
    if footer:
        table_content.append(Tfoot(footer))
    
    return Table(cls=table_cls, *args, **kwargs)(head, body)

# Try the modified UkTable function
print("\nModified UkTable output:")
print(UkTable(cols, data))

## Forms

In [ ]:
#| export
def UkFormSection(title, description, *c, button_txt='Update', outer_margin=6, inner_margin=6):
    return Div(cls=f'space-y-{inner_margin} py-{outer_margin}')(
        Div(UkH3(title), P(description, cls=TextT.medium_sm)),
        UkHSplit(), *c,
        Div(UkButton(button_txt, cls=UkButtonT.primary)) if button_txt else None)

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()